In [1]:
#!pip install torch==1.6.0
!pip install transformers==3.3.1
!pip install seqeval
!pip install fastprogress
!pip install attrdict

     |████████████████████████████████| 1.1MB 13.7MB/s 
     |████████████████████████████████| 3.0MB 51.1MB/s 
     |████████████████████████████████| 1.2MB 51.3MB/s 
     |████████████████████████████████| 890kB 49.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=3d932fca5e806ab4b152c20141e22fad465a67564a165040f3c2d933b316be5d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=64b04f8ab93c26c8879e403353b2959d49ebb420e2c9be6266f7c6f135ca0f1c
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


해당 과제의 코드는 박장원님의 "https://github.com/monologg/KoELECTRA/tree/master/finetune" 레포지토리의 코드를 변경하여 만들어졌습니다.

추신. 좋은 코드 만들어주시고, 코드 사용을 흔쾌히 허락해주신 박장원님께 감사인사 드립니다.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!cp -r /content/drive/MyDrive/keggle/boostcamp-nlp/day20 /content

In [4]:
!ls

day20  drive  sample_data


In [5]:
%cd /content/day20/

/content/day20


In [6]:
!ls

ckpt	NIKL_NE		     README_EN.md      results	   run_seq_cls.py
config	preprocessing.ipynb  README.md	       run.ipynb   run_squad.py
data	processor	     requirements.txt  run_ner.py  src


학습을 시작하기 전에 같이 동봉된 "preprocessing.ipynb"를 실행하여 나오는 "train.tsv","dev.tsv"를 "data/custom-ner"에 넣어줍니다.

In [7]:
#!!!!!!!!! need to modify tokenizer !!!!!!!!!!!!!!!
# from transformers import BertForTokenClassification
# from src import KoBertTokenizer
# !python3 run_ner.py --task custom-ner --config_file kobert.json
# tokenizer = KoBertTokenizer.from_pretrained("monologg/kobert")
# !ls -alh /content/day20/ckpt/kobert-custom-ner-ckpt/
# model = BertForTokenClassification.from_pretrained("ckpt/kobert-custom-ner-ckpt/checkpoint-16000")

In [9]:
#koelectra settings
!python3 run_ner.py --task custom-ner --config_file koelectra-base-v3.json # config/custom-ner 에서 원하는 모델의 config를 변경하고 넣어주세요.

2021-02-20 08:21:05.065423: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
02/20/2021 08:21:07 - INFO - filelock -   Lock 140092547123184 acquired on /root/.cache/torch/transformers/2cf49db823b5c8006511a6a31aa379abae71d0bad9c547a16efefabb455c4187.641239e8119fe2579734719b1edab735792dc8d889e2d8475e877b0fe81de66c.lock
Downloading: 100% 467/467 [00:00<00:00, 414kB/s]
02/20/2021 08:21:07 - INFO - filelock -   Lock 140092547123184 released on /root/.cache/torch/transformers/2cf49db823b5c8006511a6a31aa379abae71d0bad9c547a16efefabb455c4187.641239e8119fe2579734719b1edab735792dc8d889e2d8475e877b0fe81de66c.lock
02/20/2021 08:21:08 - INFO - filelock -   Lock 140091299708544 acquired on /root/.cache/torch/transformers/31f086145cb1068f7bc3631bd062bbe3cce3cb75502e79bd6cda243993c3a846.e060f8ca1da74aef5ab085f278fb3612042e7e4fc97a4917ed87540d3966ead6.lock
Downloading: 100% 263k/263k [00:00<00:00, 777kB/s]
02/20/2021 08:21:08 - INFO -

In [55]:
from transformers import ElectraTokenizer, ElectraForTokenClassification
# from src import KoBertTokenizer
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v3-discriminator") 
model = ElectraForTokenClassification.from_pretrained("ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-33000")

In [56]:
label_list=['DT-B','DT-I','LC-B','LC-I','OG-B','OG-I','PS-B','PS-I','QT-B','QT-I','TI-B','TI-I','O']

Kaggle Competition page에서 "test.txt"를 다운 받아서 "data/custom-ner"에 넣어줍니다.


In [57]:
texts=[]
with open("./data/custom-ner/test.txt", 'r') as f:
  for line in f:
    texts.append(line.rsplit("\n")[0])

In [58]:
print(len(texts))

7000


In [59]:
import torch
from tqdm.auto import tqdm

In [ ]:
labels=[]
for idx, sequence in tqdm(enumerate(texts)):
  #print(sequence)
  tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
  #print(tokens)
  inputs = tokenizer.encode(sequence, return_tensors="pt")
  #print(inputs)
  #print(len(inputs[0]))
  outputs = model(inputs)
  predictions = torch.argmax(outputs[0], dim=2)
  labels_=[]
  for i,j in zip(tokens, predictions[0].tolist()):
    #print(label_list[j])
    if i in ['[CLS]', '[SEP]']:
      continue
    if "##" in i:
      continue
    labels_.append(label_list[j])
  #print(labels_)
  labels.extend(labels_)


In [ ]:
print(len(labels))

"submission.csv"를 kaggle competition에 제출합니다.

데이터 전처리,모델 config 튜닝을 이용하여 더 좋은 성적을 얻어보세요.

In [ ]:
with open("5th_submission.csv","w") as f:
  f.write("id,tag\n")
  for idx,tag in enumerate(labels):
    f.write(str(idx))
    f.write(",")
    f.write(tag)
    f.write("\n")

In [23]:
!ls

1st_submission.csv  NIKL_NE		 README.md	   run_ner.py
ckpt		    preprocessing.ipynb  requirements.txt  run_seq_cls.py
config		    processor		 results	   run_squad.py
data		    README_EN.md	 run.ipynb	   src


In [54]:
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-33000 /content/drive/MyDrive

In [ ]:
!cp 5th_submission.csv /content/drive/MyDrive/keggle/boostcamp-nlp/day20/results/

In [46]:
!cp submission.csv /content/drive/MyDrive/keggle/boostcamp-nlp/day20/results/

In [11]:
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-10000 /content/drive/MyDrive
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-13000 /content/drive/MyDrive
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-20000 /content/drive/MyDrive
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-27000 /content/drive/MyDrive
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-30000 /content/drive/MyDrive
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-31000 /content/drive/MyDrive
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-40000 /content/drive/MyDrive


In [50]:
!cp -r ./ckpt/koelectra-base-v3-custom-ner-ckpt/checkpoint-24000 /content/drive/MyDrive